In [51]:
!pip install matplotlib
!pip install rdkit-pypi
!pip install gensim
!pip install mol2vec
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 22.8 MB/s eta 0:00:00


In [52]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import rdmolops
from mol2vec.features import mol2alt_sentence, MolSentence, sentences2vec
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from Bio.Seq import Seq  # Importa Seq de Biopython

# Cargar los datos
data = pd.read_csv('resultado_limpio.csv')  # Asegúrate de ajustar el nombre del archivo
sequences = data['Seqence'].tolist()
activities = data['Activity'].tolist()

# Convertir secuencias de aminoácidos a moléculas
def sequence_to_mol(sequence):
    # Generar un SMILES de la secuencia de aminoácidos
    # Aquí se puede usar un método para convertir la secuencia a un SMILES válido
    # A continuación se muestra un ejemplo simple, necesitarás ajustar esto a tu necesidad
    amino_acid_to_smiles = {
        'A': 'CC(C(=O)N)C(C(=O)O)N',
        'R': 'CC(C(=O)N)C(C(=O)O)N',  # Usa una representación de SMILES real
        'N': 'CC(C(=O)N)C(C(=O)O)N',  # Deberías agregar todos los aminoácidos
        # Completa el diccionario con los SMILES de cada aminoácido
    }

    # Convierte la secuencia a una cadena SMILES
    smiles = ''.join([amino_acid_to_smiles.get(aa, '') for aa in sequence if aa in amino_acid_to_smiles])
    return Chem.MolFromSmiles(smiles)  # Convierte la cadena SMILES a un objeto RDKit

# Cargar el modelo Mol2Vec
model = None
try:
    from mol2vec import Mol2Vec
    model = Mol2Vec.load('path_to_mol2vec_model.bin')  # Cambia esto por la ruta de tu modelo entrenado
except ImportError:
    print("No se pudo importar Mol2Vec. Asegúrate de que está instalado correctamente y que estás usando la versión adecuada.")

# Generar oraciones de moléculas y vectores usando mol2vec
mol_sentences = []
for seq in sequences:
    mol = sequence_to_mol(seq)  # Convierte la secuencia a un objeto RDKit
    if mol is not None and model is not None:  # Solo procesar moléculas válidas y si el modelo se cargó correctamente
        # Usar el objeto de molécula directamente
        mol_sentence = mol2alt_sentence(mol, radius=2)  # Se incluye radius
        mol_sentences.append(MolSentence(mol_sentence))

# Generar vectores a partir de oraciones moleculares
if model is not None:
    vectors = sentences2vec(mol_sentences, model)  # Ahora incluye el modelo

    # Calcular la matriz de similitud
    similarity_matrix = cosine_similarity(vectors)

    # Agrupación de Péptidos
    kmeans = KMeans(n_clusters=5)  # Ajusta el número de clusters según sea necesario
    clusters = kmeans.fit_predict(vectors)

    # Añadir los clusters al DataFrame
    data['Cluster'] = clusters

    # Análisis de la actividad por grupo
    activity_distribution = data.groupby('Cluster')['Activity'].value_counts(normalize=True)
    print(activity_distribution)

    # Dividir los datos
    X_train, X_test, y_train, y_test = train_test_split(vectors, data['Activity'], test_size=0.2, random_state=42)

    # Entrenar un clasificador
    clf = RandomForestClassifier()
    clf.fit(X_train, y_train)

    # Hacer predicciones
    y_pred = clf.predict(X_test)

    # Evaluar el modelo
    print(classification_report(y_test, y_pred))
else:
    print("No se pudo ejecutar el modelo debido a la falta de carga de Mol2Vec.")

No se pudo importar Mol2Vec. Asegúrate de que está instalado correctamente y que estás usando la versión adecuada.
No se pudo ejecutar el modelo debido a la falta de carga de Mol2Vec.
